In [53]:
from typing import Sequence, Optional

import os
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt

ABSOLUTE_PATH_SRC = "C:/Users/arman/it/prog/AI/src"
GOOD_WIDTH = 900
GOOD_HEIGHT = 600

In [2]:
def show(*imgs: Sequence[cv2.typing.MatLike]):
    counter = 1
    for img in imgs:
        width, height, *_ = img.shape
        width, height = int(width / (height / GOOD_HEIGHT)), GOOD_HEIGHT
        resize_img = cv2.resize(img, (width, height))
        cv2.imshow(f"image - {counter}", resize_img)
        counter += 1
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [77]:
def get_image():
    # return cv2.imread(f"{ABSOLUTE_PATH_SRC}/random_images/image_6.jpg", 1)
    images_names = os.listdir(f"{ABSOLUTE_PATH_SRC}/random_images/")
    return cv2.imread(f"{ABSOLUTE_PATH_SRC}/random_images/{random.choice(images_names)}", 1)

1)	Создание маски изображения (черно-белого изображения, на которой белым выделен определенный объект, который требуется сегментировать, и черным выделено все остальное) либо вручную, либо при помощи 

In [ ]:
img = get_image()

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# ret, mask = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
ret, mask = cv2.threshold(img,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

show(mask)


2)	Генератор изображений для аугментации изображения, случайным образом реализующий одну или несколько операций, таких как:

In [7]:
def scale(
    img: cv2.typing.MatLike,
    width: Optional[int] = None,
    height: Optional[int] = None
) -> cv2.typing.MatLike:
    if width is None and height is None:
        width, height = GOOD_WIDTH, GOOD_HEIGHT
    elif width is None:
        width = img.shape[0]
        width, height = int(width / (height / GOOD_HEIGHT)), GOOD_HEIGHT
    elif height is None:
        height = img.shape[1]
        width, height = GOOD_WIDTH, int(height / (width / GOOD_WIDTH))
    
    return cv2.resize(img, (width, height))

In [34]:
def move(
    img: cv2.typing.MatLike,
    square1: tuple[int, int, int, int] = [0, 0, 100, 100], # x1, y1, x2, y2
    square2: tuple[int, int, int, int] = [200, 200, 300, 300], # x1, y1, x2, y2
) -> cv2.typing.MatLike:
    first_square = (img[square1[1]:square1[3], square1[0]:square1[2]]).copy()
    second_square = (img[square2[1]:square2[3], square2[0]:square2[2]]).copy()
    img[square1[1]:square1[3], square1[0]:square1[2]] = second_square
    img[square2[1]:square2[3], square2[0]:square2[2]] = first_square
    return img

In [58]:
def rotate(
    img: cv2.typing.MatLike,
    angle: int # [0, 90]
) -> cv2.typing.MatLike:
    img_center = tuple(np.array(img.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(img_center, angle, 1.0)

    h, w = img.shape[:2]
    img_c = (w / 2, h / 2)
    rad = math.radians(angle)
    sin = math.sin(rad)
    cos = math.cos(rad)
    b_w = int((h * abs(sin)) + (w * abs(cos)))
    b_h = int((h * abs(cos)) + (w * abs(sin)))

    rot_mat[0, 2] += ((b_w / 2) - img_c[0])
    rot_mat[1, 2] += ((b_h / 2) - img_c[1])

    return cv2.warpAffine(img, rot_mat, (b_w, b_h), flags=cv2.INTER_LINEAR)


In [78]:
def reflection(
    img: cv2.typing.MatLike,
    vertical: bool = False,
    horizontal: bool = False
) -> cv2.typing.MatLike:
    if vertical and horizontal:
        return cv2.flip(img, -1)
    elif vertical:
        return cv2.flip(img, 0)
    elif horizontal:
        return cv2.flip(img, 1)
    else:
        return img

In [80]:
img = get_image()
ready_images = []
w, h = img.shape[:2]
for _ in range(100+1):
    reflection_vert = random.choice([False, True])
    reflection_hori = random.choice([False, True])

    angle = random.randint(0, 180)

    ready_images.append(reflection(rotate(img, angle=angle), reflection_vert, reflection_hori))
    
show(ready_images[0])